In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
train = pd.read_csv('../input/tabular-playground-series-aug-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-aug-2021/test.csv')
sub = pd.read_csv('../input/tabular-playground-series-aug-2021/sample_submission.csv')

In [ ]:
train.describe()

In [ ]:
# for i in train:
#     print(train[i].min())

In [ ]:
Ids = test['id']

In [ ]:
test.drop('id', inplace=True, axis=1)

In [ ]:
train.isnull().sum()

In [ ]:
y=train['loss']

In [ ]:
train.drop('loss', inplace=True, axis=1)

In [ ]:
train.drop('id', inplace=True, axis=1)

In [ ]:
train.shape

In [ ]:
# test.describe()

In [ ]:
## scikitlearn's Elasticnet Performed well

In [ ]:
from sklearn.linear_model import ElasticNet
# regr = ElasticNet(random_state=42, max_iter=2000,)
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error




def objective(trial,train, y):
#     print(y[:10])
    X_train, X_test, y_train,y_test = train_test_split(train, y, test_size = 0.3, random_state = 12)
    
    alph=trial.suggest_float("alpha", 0, 0.3),
    
    alph = float('.'.join(str(ele) for ele in alph))

    l1_rat=trial.suggest_float("l1_ratio",0.0,0.2),
    
    l1_rat = float('.'.join(str(ele) for ele in l1_rat))

    
#     normal= trial.suggest_categorical('normalize',[True,False]),
    max_it=trial.suggest_int("max_iter", 100,5000),
    max_it= int('.'.join(str(ele) for ele in max_it))

#     posit= trial.suggest_categorical('positive',[True,False]),
    regr=ElasticNet(alpha=alph, l1_ratio=l1_rat)
#     alpha=alph, l1_ratio=l1_rat, normalize=normal, max_iter=max_it, positive=posit
    regr.fit(X_train,y_train)
    
    y_pred=regr.predict(X_test)
    print(y_pred[:5])
    error=mean_squared_error(y_test, y_pred)
    
    
    return(error)



In [ ]:
study1 = optuna.create_study(direction='minimize')
study1.optimize(lambda trial: objective(trial, train, y), n_trials=100,show_progress_bar=True)

In [ ]:
study1.best_params

In [ ]:
study2 = optuna.create_study(direction='minimize')
study2.optimize(lambda trial: objective(trial, train, y), n_trials=100,show_progress_bar=True)

In [ ]:
study2.best_params

In [ ]:
regrs=ElasticNet(alpha=0, l1_ratio=0.1, max_iter=3500)
#     alpha=alph, l1_ratio=l1_rat, normalize=normal, max_iter=max_it, positive=posit
regrs.fit(train,y)

In [ ]:
regrs2=ElasticNet(**study2.best_params)
#     alpha=alph, l1_ratio=l1_rat, normalize=normal, max_iter=max_it, positive=posit
regrs2.fit(train,y)

In [ ]:
loss=regrs2.predict(test)

In [ ]:
# from sklearn.ensemble import GradientBoostingRegressor

# booster = GradientBoostingRegressor( loss='ls', learning_rate=0.001)
# booster.fit(train, y)

In [ ]:
# loss=booster.predict(test)

In [ ]:
# from sklearn.tree import DecisionTreeRegressor
# dt=DecisionTreeRegressor()

# dt.fit(train,y)

###Performed worst

In [ ]:
# loss=dt.predict(test)

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

X_train, X_test, y_train,y_test = train_test_split(train, y, test_size = 0.2, random_state = 12)
# we are converting the training and testing data into xgboost optimized matrix for efficiency

dtrain = xgb.DMatrix(X_train,y_train)
dtest  = xgb.DMatrix(X_test,y_test)

In [ ]:
import re
def return_rmse(params):
    model = xgb.train(params , dtrain, evals = [(dtest, 'eval')],
          early_stopping_rounds=10,verbose_eval = 0)
    result = model.eval(dtest)
    result = np.float(re.search(r'[\d.]+$',result).group(0))
    print(result)
    return(result)

In [ ]:
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler



def objective(trial):
   
    param = {
#                 "n_estimators" : trial.suggest_int('n_estimators', 0, 1000),
                'max_depth':trial.suggest_int('max_depth', 3, 15),
                'reg_alpha':trial.suggest_uniform('reg_alpha',0,10),
                'reg_lambda':trial.suggest_uniform('reg_lambda',0,5),
                'min_child_weight':trial.suggest_int('min_child_weight',0,5),
                'gamma':trial.suggest_uniform('gamma', 0, 4),
                'learning_rate':trial.suggest_loguniform('learning_rate',0.001,0.5),
                'colsample_bytree':trial.suggest_uniform('colsample_bytree',0.4,0.9),
                'subsample':trial.suggest_uniform('subsample',0.4,0.9),

                'nthread' : -1
            }
    return(return_rmse(param)) # this will return the rmse score 

In [ ]:
study3 = optuna.create_study(direction='minimize',sampler=TPESampler())
study3.optimize(objective, n_trials= 100,show_progress_bar = True)

In [ ]:
study3.best_params

In [ ]:
fig = optuna.visualization.plot_optimization_history(study3)
fig.show()

In [ ]:
import xgboost as xgb


xgbr = xgb.XGBRegressor()
xgbr.fit(train,y)

In [ ]:
# max_depth=11,reg_alpha=0.8173816703689041,reg_lambda=3.7499802859415343,min_child_weight=5,gamma= 3.579186100259433,learning_rate= 0.012829888602199139,colsample_bytree=0.7526602405729004,subsample=0.6138792615327984

In [ ]:
loss=xgbr.predict(test)

In [ ]:
loss=pd.DataFrame(loss, columns=['loss'])


In [ ]:
loss

In [ ]:
y.hist()

In [ ]:
loss.hist()

In [ ]:
res=pd.concat([Ids,loss],axis=1)

In [ ]:
res

In [ ]:
sub = res.to_csv('submission.csv', index=False)